In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [6]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [7]:
nlp = spacy.load('en_core_web_sm')

# Load dataset

In [8]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [9]:
# number of documents
len(docs)

18846

In [10]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [11]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 18846/18846 [10:38<00:00, 29.52it/s]


number of removed short documents: 3975
total number of tokens: 1442761
number of tokens to be removed: 417459
number of additionally removed short documents: 2185
total number of tokens: 1000435

minimum word count number: 15
this number can be less than MIN_COUNTS because of document removal


In [12]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [13]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

12686it [00:01, 7514.94it/s]


In [14]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [15]:
# number of windows (equals to the total number of tokens)
data.shape[0]

1000435

# Get unigram distribution

In [16]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [17]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

Wall time: 9min 27s


In [18]:
# number of unique words
vocab_size

7863

# Prepare initialization for document weights

In [19]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [20]:
%%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

Wall time: 8.44 s


In [21]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : food taste eat MSG lot Serbs sure little place Muslims
topic 1 : fire FBI shoot kill tank weapon Koresh attack report child
topic 2 : law public President encryption gun state firearm private job chip
topic 3 : New cover appear book price Turkey Press copy York art
topic 4 : pin latch modem input jumper logic current slave Symbol connect
topic 5 : probably idea lot chip sure pretty far hear order ago
topic 6 : University gun article Wright Professor rate quote study crime James
topic 7 : research science age group Center material National service scientific conference
topic 8 : evidence reason cause mind bad life claim happen answer course
topic 9 : space launch Earth orbit NASA planet satellite mission solar Space
topic 10 : Mary probably great happen bad far sure Latest end place
topic 11 : bike battery engine bad buy sure ride bit light little
topic 12 : man Jesus Christ Lord armenian turkish Armenians child greek sin
topic 13 : card disk driver color Windows software boar

In [22]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 12686/12686 [00:07<00:00, 1679.65it/s]


# Save data

In [23]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)